[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/Klabauterkerl/finnish-chopper/blob/main/fairseq_bpe.ipynb)

In [ ]:
%pip install fairseq
%pip install sacrebleu sentencepiece
%pip install tensorboardX

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define paths for the mounted Google Drive
base_path = "/content/drive/MyDrive/translation_model"
dataset_path = f"{base_path}/dataset"
!mkdir -p "{dataset_path}"
data_bin_path = f"{base_path}/data-bin"
checkpoints_path = f"{base_path}/checkpoints"
logs_path = f"{base_path}/logs"

In [ ]:
# Download and extract dataset
!wget -P "{dataset_path}" https://www.statmt.org/europarl/v9/training/europarl-v9.fi-en.tsv.gz
!gunzip "{dataset_path}/europarl-v9.fi-en.tsv.gz"


In [ ]:
# Split dataset into two files, each containing one column of the original dataset
!cut -f1 {dataset_path}/europarl-v9.fi-en.tsv > {dataset_path}/europarl-v9.fi
!cut -f2 {dataset_path}/europarl-v9.fi-en.tsv > {dataset_path}/europarl-v9.en

In [ ]:
# Install Moses for preprocessing
!git clone https://github.com/moses-smt/mosesdecoder.git

# Normalize & Tokenize Finnish and English texts
!cat {dataset_path}/europarl-v9.fi | mosesdecoder/scripts/tokenizer/normalize-punctuation.perl fi |\
mosesdecoder/scripts/tokenizer/tokenizer.perl -threads 8 -a -l fi\
> {dataset_path}/europarl-v9.tok.fi
!cat {dataset_path}/europarl-v9.en | mosesdecoder/scripts/tokenizer/normalize-punctuation.perl en |\
mosesdecoder/scripts/tokenizer/tokenizer.perl -threads 8 -a -l en \
> {dataset_path}/europarl-v9.tok.en

!perl mosesdecoder/scripts/training/clean-corpus-n.perl {dataset_path}/europarl-v9.tok fi en \
    {dataset_path}/tokenized.tok.clean 1 50 

In [ ]:
# Install subword-nmt for BPE-encoding
%pip install subword-nmt

# Create BPE directory and set its path
bpe_path = f"{dataset_path}/bpe"
!mkdir -p "{bpe_path}"

# Learn a joint BPE model and vocabulary
!subword-nmt learn-joint-bpe-and-vocab \
     --input {dataset_path}/tokenized.tok.clean.fi {dataset_path}/tokenized.tok.clean.en -s 32000 \
     -o {dataset_path}/bpe.codes --write-vocabulary {dataset_path}/vocab.fi {dataset_path}/vocab.en
# Apply the learned BPE model and vocabulary
!subword-nmt apply-bpe -c {bpe_path}/bpe.codes \
     --vocabulary {bpe_path}/vocab.fi < {bpe_path}/europarl-v9.fi.tok > {bpe_path}/europarl-v9.bpe.fi
!subword-nmt apply-bpe -c {bpe_path}/bpe.codes \
     --vocabulary {bpe_path}/vocab.en < {bpe_path}/europarl-v9.en.tok > {bpe_path}/europarl-v9.bpe.en

In [ ]:
!head -n 10000 {bpe_path}/europarl-v9.bpe.fi > {bpe_path}/test.bpe.fi
!tail -n +10001 {bpe_path}/europarl-v9.bpe.fi | head -n 10000 > {bpe_path}/valid.bpe.fi
!tail -n +20001 {bpe_path}/europarl-v9.bpe.fi > {bpe_path}/train.bpe.fi

!head -n 10000 {bpe_path}/europarl-v9.bpe.en > {bpe_path}/test.bpe.en
!tail -n +10001 {bpe_path}/europarl-v9.bpe.en | head -n 10000 > {bpe_path}/valid.bpe.en
!tail -n +20001 {bpe_path}/europarl-v9.bpe.en > {bpe_path}/train.bpe.en

In [ ]:
# Define paths for the mounted Google Drive
base_path = "/content/drive/MyDrive/translation_model"
data_bin_path = f"{base_path}/data-bin"
checkpoints_path = f"{base_path}/checkpoints"
dataset_path = f"{base_path}/dataset"

# Create directories in Google Drive
!mkdir -p "{data_bin_path}"
!mkdir -p "{checkpoints_path}"

In [ ]:
# Create Dataset using BPE Data
!fairseq-preprocess --source-lang fi --target-lang en \
    --trainpref {dataset_path}/train.bpe --validpref {dataset_path}/valid.bpe --testpref {dataset_path}/test.bpe \
    --destdir {data_bin_path}/bpe --joined-dictionary --workers 20

In [ ]:
# Train Model using BPE Dataset
!fairseq-train {data_bin_path}/bpe \
    --arch transformer --share-all-embeddings \
    --encoder-layers 5 --decoder-layers 5 \
    --encoder-embed-dim 512 --decoder-embed-dim 512 \
    --encoder-ffn-embed-dim 2048 --decoder-ffn-embed-dim 2048 \
    --encoder-attention-heads 8 --decoder-attention-heads 8 \
    --dropout 0.1 --attention-dropout 0.1 --relu-dropout 0.1 \
    --optimizer adam --lr 0.0005 --lr-scheduler inverse_sqrt \
    --warmup-updates 4000 --warmup-init-lr 1e-07 \
    --stop-min-lr 1e-09 --clip-norm 0.0 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --weight-decay 0.0001 --max-tokens 4096 \
    --update-freq 1 --max-epoch 30 --save-interval 1 \
    --keep-last-epochs 5 --log-format simple --log-interval 100 \
    --tensorboard-logdir {logs_path} --seed 42 \
    --save-dir {checkpoints_path}/bpe \
    --amp

In [ ]:
# Generate translations using BPE trained model
! fairseq-generate {data_bin_path}/bpe \
    --path {checkpoints_path}/bpe/checkpoint_best.pt \
    --beam 5 --lenpen 1.2 \
    --gen-subset test \
    --remove-bpe > {base_path}/translations_bpe.txt

In [ ]:
# Compute BLEU score
!grep ^H {base_path}/translations.txt | cut -f3- > {base_path}/hyp.txt
!grep ^T {base_path}/translations.txt | cut -f2- > {base_path}/ref.txt
!mosesdecoder/scripts/generic/multi-bleu.perl {base_path}/ref.txt < {base_path}/hyp.txt